In [1]:
%load_ext autoreload
%autoreload 2

import cryogrid_data_fetcher as cdf

2024-09-11 15:27:52 | VERBOSE  | Logger level changed to VERBOSE
2024-09-11 15:27:52 | INFO     | Registering Dask progress bar


## Load configurations

In [39]:
fname_data_config = './requests/cryogrid_pamir_region.yml'
fname_xls_config = '../cryogrid-pamir/abramov/experiments/simplest/simplest.xlsx'

experiment_bbox = 71.481314, 39.508578, 72.000418, 39.77

data_config = cdf.config.load(fname_data_config)
cryogrid_xls = cdf.config.CryoExcel(fname_xls_config, bbox_WSEN=experiment_bbox)

2024-09-11 15:52:08 | SUCCESS  | Loaded request from ./requests/cryogrid_pamir_region.yml
2024-09-11 15:52:08 | VERBOSE  | Request bbox: [70.0, 37.0, 75.0, 41.0]
2024-09-11 15:52:08 | VERBOSE  | Request dates: [1960 : 2023]
2024-09-11 15:52:08 | SUCCESS  | S3 bucket credentials found in environment variables
2024-09-11 15:52:08 | INFO     | Found root path: /Users/luke/SDSC/CryoGrid/cryogrid-pamir/abramov
2024-09-11 15:52:08 | INFO     | Loaded CryoGrid Excel configuration file: /Users/luke/SDSC/CryoGrid/cryogrid-pamir/abramov/experiments/simplest/simplest.xlsx
2024-09-11 15:52:08 | INFO     | Bounding box (WSEN): (71.481314, 39.508578, 72.000418, 39.77)
2024-09-11 15:52:08 | INFO     | Start and end times: 1970-01-01 - 2020-12-31


## DEM

In [35]:
dem = cdf.download_dem_from_planetary_computer(data_config)
dem = dem.rio.clip_box(*experiment_bbox, crs='EPSG:4326')

dem.rio.to_raster(cryogrid_xls.fname.dem)

2024-09-11 15:44:53 | INFO     | Loading DEM from s3://spi-pamir-cryogrid/pamir/dem/DEM-cop-dem-glo-30-W7000_S3700_E7500_N4100.zarr


Xarray processing:   0%|          | 0/12 [00:00<?, ?it/s]

## Stratigraphy

In [40]:
ds_strat = (
    (dem * 0 + 1)  # setting stratigraphy to 1 for simplest case
    .astype(int)
    .rename(elevation='stratigraphy')
    .assign_attrs(
        band='stratigraphy',
        platform='cryogrid-dummy',
        source='manual'))

ds_strat.attrs.pop("proj:epsg", None)
ds_strat.attrs.pop("proj:shape", None)

strat_fname = cryogrid_xls.get_class_filepath('READ_DATASET', index=1)
ds_strat.rio.to_raster(strat_fname)

Xarray processing:   0%|          | 0/24 [00:00<?, ?it/s]

## ERA5 Forcing

In [37]:
s3_path = data_config.era5.dst_dir_s3 

cdf.era5.to_matlab.main(
    s3_path, 
    cryogrid_xls.fname.era5, 
    bbox_WSEN=cryogrid_xls.bbox_WSEN, 
    year_start=cryogrid_xls.time.time_start.year, 
    year_end=cryogrid_xls.time.time_end.year)

2024-09-11 15:46:01 | VERBOSE  | Loading ERA5 data from s3://spi-pamir-cryogrid/pamir/era5/lvl_500_1000


Xarray processing:   0%|          | 0/1538 [00:00<?, ?it/s]

2024-09-11 15:46:57 | VERBOSE  | Clipping to bbox (71.481314, 39.508578, 72.000418, 39.77) and years 1970 to 2020


Xarray processing:   0%|          | 0/25707 [00:00<?, ?it/s]

2024-09-11 15:48:44 | VERBOSE  | Saving clipped data to /Users/luke/SDSC/CryoGrid/cryogrid-pamir/abramov/forcing/cryogrid-pamir_abramov-era5-1970_2020.nc
2024-09-11 15:48:52 | SUCCESS  | Saved ERA5 to /Users/luke/SDSC/CryoGrid/cryogrid-pamir/abramov/forcing/cryogrid-pamir_abramov-era5-1970_2020.mat


PosixPath('/Users/luke/SDSC/CryoGrid/cryogrid-pamir/abramov/forcing/cryogrid-pamir_abramov-era5-1970_2020.mat')